# 4.3 Accessing metadata via Python from a Jupyter Notebook
Now that we know how searching for metadata works through geoportals, we want to look a bit more behind the HTML interface. When you're filtering the datasets there are requests executed on the CSW in the background. The construction of the requests is always the same at CSWs. But before we can start let's quickly set up our all other components which we need. The OGC have an own Python library, called OWSLib, for client programming. This library allows the users to access and utilize geospatial data from their online services, like WMS or CSW, via Python. After installing the library you will also need to import the CSW class from OWSLib and some methods for later use. 

In [42]:
%pip install OWSLib
from owslib.csw import CatalogueServiceWeb
from owslib.fes import PropertyIsEqualTo, BBox


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\Tobias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Now we need the URL from the INSPIRE CSW. I've you search for the "INSPIRE CSW" in your brwoser, you should get the the catalogue service in one of the first links (https://inspire-geoportal.ec.europa.eu/GeoportalProxyWebServices/resources/OGCCSW202?request=GetCapabilities&service=CSW). It's the GetCapabilities request from the CSW. In the GetCapabilities you get all the metadata for the catalogue service.
But to use the catlogue service from INSPIRE in the CSW class from OWSLib you only need the first part of the URL until the "?". 

In [40]:
csw = CatalogueServiceWeb('https://inspire-geoportal.ec.europa.eu/GeoportalProxyWebServices/resources/OGCCSW202')
[op.name for op in csw.operations]
bbox_query = BBox([52.244094, 9.212036, 52.574420, 10.318909])
csw.getrecords2(constraints=[bbox_query])
csw.results

{'matches': 385, 'returned': 10, 'nextrecord': 11}

In [53]:
record = csw.getrecordbyid(id=['412966c0-a49f-40d2-8419-d7eca338da0f'])

In [44]:
csw.getdomain('GetRecords.resultType')
csw.results

XMLSyntaxError: Space required after the Public Identifier, line 1, column 50 (<string>, line 1)